# Director Analysis
Use the IMDb data to uncover the film directors' career path
In here, I use the IMDb set that includes millions films' information which has already been cleaned for use. The data clean steps will be post later.
Firstly, load dataset:

In [8]:
#Module used in the data
import pandas as pd
import numpy as np
from   scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import time
import psycopg2
from tqdm import tqdm
pd.set_option('display.max_columns', 20)
pd.set_option('precision',2)


In [30]:
def tempfileread():
    df = pd.read_csv('/Users/xintongli/PycharmProjects/Project/Film Data Analysis/SQL_data.csv',
                     index_col = 0,low_memory=False)
    df_name = pd.read_csv('/Users/xintongli/PycharmProjects/Project/Film Data Analysis/SQL_name.csv',
                          index_col = 0)
    #Depend on specific which table to analyze
    df_flexible = pd.read_csv('/Users/xintongli/PycharmProjects/Project/Film Data Analysis/SQL_exp.csv',
                          index_col = 0)
    summary = pd.read_csv('/Users/xintongli/PycharmProjects/Project/Film Data Analysis/SQL_summary.csv',
                          index_col = 0)
    dflist = pd.read_csv('/Users/xintongli/PycharmProjects/Project/Film Data Analysis/List_temp')
    return df, df_name, df_flexible, summary, dflist
df      = tempfileread()[0].sort_index()
df_name = tempfileread()[1].sort_index()
year_a  = tempfileread()[2].sort_index()
summary = tempfileread()[3].sort_index()


df: data sort by film title
df_name: uses the film directors as entry, described each film directors' profile:
    nconst (string) - alphanumeric unique identifier of the name/person
    primaryName (string)– name by which the person is most often credited
    birthYear
    deathYear
    age
year_a: year analysis. Analyze year related data:
    
summary: general information about data
    numberall - numbers of directed film
    runtimeminutes - average film running times of each director
    averagerating - arithmetic average of rating
    numvotes - total number of people who vote for film, indicating the popularity of the film/director

Take a look at general information

In [32]:
summary['numberall'].describe()

count    111380.00
mean          2.56
std           4.98
min           1.00
25%           1.00
50%           1.00
75%           2.00
max         202.00
Name: numberall, dtype: float64

In [37]:
df[['runtimeminutes', 'averagerating', 'numvotes']].describe()

,runtimeminutes,averagerating,numvotes
count,256399.00,285546.00,2.86e+05
mean,94.55,6.15,3.30e+03
std,159.15,1.33,3.16e+04
min,1.00,1.00,5.00e+00
25%,81.00,5.40,1.40e+01
50%,91.00,6.30,4.90e+01
75%,102.00,7.00,2.58e+02
max,51420.00,10.00,2.30e+06


In [39]:
print('Percentil of direct 1 film: ' +'{}'.format(stats.percentileofscore(summary['numberall'].to_list(),2)))
print('Percentil of direct 5 film: ' +'{}'.format(stats.percentileofscore(summary['numberall'].to_list(),5)))
print('The longest film is ' + '{}'.format(df['runtimeminutes'].max()) + ', number votes ' + 
      '{}'.format(df['numvotes'][df['runtimeminutes'] == df['runtimeminutes'].max()].iloc[0]))
print('Percentage of lower than 100 votes ' +'{}'.format(stats.percentileofscore(df['numvotes'].to_list(),100)))


Percentil of direct 1 film: 70.35015263063387
Percentil of direct 5 film: 90.27697970910397
The longest film is 51420.0, number votes 49
Percentage of lower than 100 votes 62.08842007942678


There are 111380 film directors in the summary. 70.4% of them direct for only 1 film, and 10% directs more than 5 movies.
Among 200 thousands films, the length of film is quite stable as well, as they are around 1.5 hours. Interestingly, an outlier -- the 51420 minutes(35 days) film does exists and 49 users vote for that. The rating of IMDb ranged from 1 to 10, with the mean score 6.15, and 75% of rates is lower than 7. Get high scores is not easy in IMDb. The popularity are different for film and directors. 62.1% of films have fewer than 100 people votes.
This is the overview of the film data. Those statistics indicates that a large part of films are produced without attracting the public's attention, and some films are made for different reason, as the 35-days-long documentary  definitely targets to be wathed as common movie does.